In [1]:
import sys
sys.path.append('./airflow')

import os
from functions.images.detr.util.features import get_detr_model, extract_detr_features
from functions.images.yolo.util.features import extract_yolo_features
from functions.images.detr.util.transform import reshape

import pymongo
import json
import requests

In [2]:
with open("./airflow/config/env.json", "r") as file:
    config = json.load(file)
    mongo_url = config['mongodb']['MONGO_ATLAS_PYTHON_GCP']

In [11]:
with pymongo.MongoClient(mongo_url) as client:
    db = client['imcp']
    
    pipeline = [{
            '$sort': {'url': 1}
        }, {
            '$project': {'created_time': 0, 'publisher': 0, '_id': 0 }
        }, {
            '$limit': 8000
        }
    ]
    
    documents = db['refined'].aggregate(pipeline)
    
    for doc in documents:
        print(doc['url'])
        print(doc.keys())
        break

http://images.cocodataset.org/train2017/000000000009.jpg
dict_keys(['url', 'caption', 'short_caption', 'caption_tokens', 'short_caption_tokens'])


In [5]:
# Tải mô hình DETR
model, postprocessor = get_detr_model(pretrained=True)

# Khởi tạo một dictionary để lưu trữ các đặc trưng của ảnh
image_features = {}

# Lặp qua từng ảnh trong thư mục
with pymongo.MongoClient(mongo_url) as client:
    db = client['imcp']
    pipeline = [{
            '$sort': {'url': 1}
        }, {
            '$project': {'created_time': 0, 'publisher': 0, '_id': 0 }
        }, {
            '$limit': 8000
        }
    ]
    documents = db['refined'].aggregate(pipeline)
    
    for doc in documents:
        response = requests.get(doc['url'])
        # # Trích xuất đặc trưng từ YOLO
        # yolo_features = extract_yolo_features(img_path)
        
        # Tải và chuyển đổi hình ảnh đầu vào cho DETR
        image_tensor = reshape(response.content)
        
        # Trích xuất đặc trưng từ DETR
        detr_features = extract_detr_features(image_tensor, model)
        doc['detr_encode'] = detr_features
        print(doc.keys())
        print(doc['detr_encode'].shape)
        break

d:\Programming\imcp-integration\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Programming\imcp-integration\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


dict_keys(['url', 'caption', 'short_caption', 'caption_tokens', 'short_caption_tokens', 'detr_encode'])
(400,)


In [ ]:
# %pip install scipy